In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime

In [2]:
# Read country file and store into Pandas data frame
land_temp_by_country_csv = "../../ProjectOne_datasets/GlobalLandTemperaturesByCountry.csv"
land_temp_by_country_df = pd.read_csv(land_temp_by_country_csv)
land_temp_by_country_df.tail()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe
577461,2013-09-01,NaN,NaN,Zimbabwe


In [3]:
#Load Population Growth by Country dataset
population_growth_country_csv = "../../ProjectOne_datasets/population-by-country.csv"
population_growth_country_df = pd.read_csv(population_growth_country_csv)
population_growth_country_df.tail()

#Rename Column for World Population data
population_growth_country_df = population_growth_country_df.rename(columns={"Population by Country (Clio Infra (2016))":"Population",
                                                                           "Entity":"Country"})
population_growth_country_df.head()

,Country,Code,Year,Population
0,Afghanistan,AFG,1500,2000000.0
1,Afghanistan,AFG,1600,2500000.0
2,Afghanistan,AFG,1700,2500000.0
3,Afghanistan,AFG,1800,3280000.0
4,Afghanistan,AFG,1820,3280000.0


In [4]:
print(f"Country Data Size: {len(land_temp_by_country_df)}")


Country Data Size: 577462


In [5]:
# Cleanup Datasets
#Drop AverageTemperatureUncertainty
land_temp_by_country_df = land_temp_by_country_df.drop(["AverageTemperatureUncertainty"], axis=1)

#Remove rows with null temperatures
land_temp_by_country_df = land_temp_by_country_df[land_temp_by_country_df["AverageTemperature"].notna()]

print(f"Country Data Size: {len(land_temp_by_country_df)}")

Country Data Size: 544811


In [6]:
#Retrieve rows for the last century
cent_land_temp_by_country_df = land_temp_by_country_df.loc[land_temp_by_country_df["dt"] >= "1900-01-01"]

print(f"Country Size: {len(cent_land_temp_by_country_df)}")


Country Size: 328818


In [7]:
cent_land_temp_by_country_df.dtypes

dt                     object
AverageTemperature    float64
Country                object
dtype: object

In [8]:
global countries_temperatures_df 
columns = ["Decade", "Land Temperature", "Country"]
countries_temperatures_df = pd.DataFrame(columns = columns)

def get_surface_temps_by_country(country):
    global countries_temperatures_df 
    temp_df = cent_land_temp_by_country_df.loc[land_temp_by_country_df["Country"]==country]
    temp_df = temp_df.groupby((pd.DatetimeIndex(temp_df["dt"]).year//10)*10).mean()
    temp_df["Country"] = country

    #Reset index, and rename columns appropriately
    temp_df.reset_index(inplace=True)
    temp_df = temp_df.rename(columns = {'dt':'Decade'})
    temp_df = temp_df.rename(columns = {'AverageTemperature':'Land Temperature'})
    temp_df = temp_df.loc[temp_df["Decade"] != 2010]
    temp_df
    countries_temperatures_df = pd.concat([countries_temperatures_df, temp_df], axis=0)

In [9]:
global countries_population_df 
columns = ["Decade", "Population", "Country"]
countries_population_df = pd.DataFrame(columns = columns)

def get_population_by_country(country):
    global countries_population_df 
    
    temp_df = population_growth_country_df.loc[population_growth_country_df["Country"] == country]
    temp_df = temp_df.loc[temp_df["Year"] >= 1900]
    temp_df = temp_df.rename(columns = {'Year':'Decade'})
    temp_df = temp_df.drop(["Code"], axis=1)
    countries_population_df = pd.concat([countries_population_df, temp_df], axis=0)

In [11]:
list_of_countries = ["United States", "China", "Brazil", "Denmark", "India"]

for country in list_of_countries:
    get_surface_temps_by_country(country)
    get_population_by_country(country)

print(f"Temperature lenght: {len(countries_temperatures_df)}")
print(f"Temperature lenght: {len(countries_population_df)}")     


Temperature lenght: 55
Temperature lenght: 55


In [12]:
#Final cleansed and merged datasets
final_df = pd.merge(countries_temperatures_df, countries_population_df, on=["Decade", "Country"])

# final_df = final_df.drop(["Code"], axis=1)
# final_df = final_df.drop(["Country_y"], axis=1)
# final_df = final_df.rename(columns = {'Country_x':'Country'})

print(f"Merged Data Set lenght: {len(final_df)}") 
print(final_df)



Merged Data Set lenght: 55
   Decade  Land Temperature        Country    Population
0    1900          8.448225  United States  7.639100e+07
1    1910          8.460250  United States  9.276700e+07
2    1920          8.755250  United States  1.068810e+08
3    1930          9.086992  United States  1.236680e+08
4    1940          8.924092  United States  1.326370e+08
5    1950          8.913233  United States  1.522710e+08
6    1960          8.721917  United States  1.806710e+08
7    1970          8.696925  United States  2.050520e+08
8    1980          9.104308  United States  2.277265e+08
9    1990          9.317558  United States  2.501319e+08
10   2000          9.646725  United States  2.821580e+08
11   1900          6.320558          China  4.000000e+08
12   1910          6.320958          China  4.230000e+08
13   1920          6.621258          China  4.720000e+08
14   1930          6.590158          China  4.890000e+08
15   1940          7.002717          China  5.187700e+08
16  

In [13]:
#Reorganize the column names for ease of redability
column_names = ["Country", "Decade", "Land Temperature", "Population"]
final_df = final_df.reindex(columns=column_names)
final_df

,Country,Decade,Land Temperature,Population
0,United States,1900,8.448225,7.639100e+07
1,United States,1910,8.460250,9.276700e+07
2,United States,1920,8.755250,1.068810e+08
3,United States,1930,9.086992,1.236680e+08
4,United States,1940,8.924092,1.326370e+08
5,United States,1950,8.913233,1.522710e+08
6,United States,1960,8.721917,1.806710e+08
7,United States,1970,8.696925,2.050520e+08
8,United States,1980,9.104308,2.277265e+08
9,United States,1990,9.317558,2.501319e+08


In [14]:
# Push the remade DataFrame to a new CSV file
final_df.to_csv("../Output/USA_Dataset.csv",
                  encoding="utf-8", index=False, header=True)